### Testing vsearch mergepairs options

In [1]:
import ipyrad as ip
print ip.__version__

DEBUG:ipyrad:H4CKERZ-mode: __loglevel__ = DEBUG


0.0.66


In [2]:
data = ip.Assembly("mergetest")

INFO:ipyrad.core.parallel:Local connection to 4 engines


New Assembly object `mergetest` created
ipyparallel setup: Local connection to 4 engines.


In [3]:
data.set_params(1, "mergetest")
data.set_params(4, "~/Documents/longi_test/longiflora/fastq/L20090356_*")
#data.set_params(5, ('TGCWG', ""))
data.set_params(5, ('TGCAG', ""))
data.set_params(17, 2)
data.set_params(10, 'pairgbs')


In [4]:
data.link_fastqs(force=True)#, append=True)

1 new Samples created in `mergetest`.
1 fastq files linked to 1 new Samples.


In [12]:
data.samples["L20090356"].files["merged"] = "mergefile"
print data.samples["L20090356"].files

{'unmapped_reads': [], 'database': [], 'consens': [], 'depths': [], 'edits': [('/home/deren/Documents/ipyrad/tests/mergetest/mergetest_edits/L20090356_R1_.fastq.gz', '/home/deren/Documents/ipyrad/tests/mergetest/mergetest_edits/L20090356_R2_.fastq.gz')], 'mapped_reads': [], 'fastqs': [('/home/deren/Documents/longi_test/longiflora/fastq/L20090356_R1_.gz', '/home/deren/Documents/longi_test/longiflora/fastq/L20090356_R2_.gz')], 'clusters': [], 'merged': 'mergefile'}


In [6]:
data.get_params()

  1   working_directory           mergetest                                    
  2   raw_fastq_path              ./*.fastq                                    
  3   barcodes_path               ./*.barcodes.txt                             
  4   sorted_fastq_path           /home/deren/Documents/longi_test/longiflora/fastq/L20090356_*
  5   restriction_overhang        ('TGCAG', '')                                
  6   max_low_qual_bases          5                                            
  7   engines_per_job             4                                            
  8   mindepth_statistical        6                                            
  9   mindepth_majrule            6                                            
  10  datatype                    pairgbs                                      
  11  clust_threshold             0.85                                         
  12  minsamp                     4                                            
  13  max_shared_heteroz

In [ ]:
data.step2(force=True)
print data.stats

In [16]:
g = []
try:
    g.get()
except AttributeError:
    print 'hi'

hi


In [ ]:
import os
import subprocess

def merge_fastq_pairs(data, sample):
    """ Merge paired fastq reads. """
    cmd = data.vsearch \
         +" --fastq_mergepairs "+sample.files.edits[0][0] \
         +" --reverse "+sample.files.edits[0][1] \
         +" --fastaout "+os.path.join(data.dirs.edits, sample.name+".merge") \
         +" --fastaout_notmerged_fwd "+os.path.join(data.dirs.edits, sample.name+".R1.notmerge") \
         +" --fastaout_notmerged_rev "+os.path.join(data.dirs.edits, sample.name+".R2.notmerge") \
         +" --fastq_allowmergestagger" \
         +" --fastq_maxdiffs "+str(4) \
         +" --fastq_maxns"+str(data.paramsdict["max_low_qual_bases"]) \
         +" --fastq_minlen"+str(32) 
    print cmd
    subprocess.check_output(cmd)


In [ ]:
 data.samples["L20090356"].files.edits[0][1]

In [ ]:
data.vsearch

In [ ]:
merge_fastq_pairs(data, data.samples["L20090356"])


In [ ]:
data.samples['L20090356'].barcode

In [ ]:
import ipyparallel as ipp

In [ ]:
client = ipp.Client()

In [ ]:
lbview = client.load_balanced_view()
result = lbview.map(sum, [[2,3],[4,5],[6,7],[8,9],3,'a'])

In [ ]:
result.get()

In [ ]:
import logging

In [ ]:
logging.StreamHandler